In [3]:
import os
import random

In [4]:
class gate_dlinkedList:
    def __init__(self, gate_id):
        self.gate_id = gate_id
        self.lock = False
        self.next = None
        self.prev = None
    
    def set_next(self, next_gate):
        self.next = next_gate
        if next_gate:
            next_gate.prev = self
        return self.next
    
    def remove_gate(self):
        if self.prev:
            self.prev.next = self.next
        if self.next:
            self.next.prev = self.prev
        self.next = None
        self.prev = None

    def __str__(self):
        str_output = f'Gate ID: {self.gate_id}\n'
        str_output += f'Lock Status: {self.gate_id}\n'
        if self.next:
            str_output += f'Next Gate: {(self.next).gate_id}\n'
        if self.prev:
            str_output += f'Prev Gate: {(self.prev).gate_id}\n'

        return str_output

In [5]:
gate1 = gate_dlinkedList(1)
gate2 = gate_dlinkedList(2)
gate3 = gate_dlinkedList(3)
# print(str(gate1))
print(str(gate1))
curr_gate = gate1.set_next(gate2)

print(str(curr_gate))
last_gate = gate2.set_next(gate3)
# print(str(gate1))
print(str(last_gate))

print(gate3)

Gate ID: 1
Lock Status: 1

Gate ID: 2
Lock Status: 2
Prev Gate: 1

Gate ID: 3
Lock Status: 3
Prev Gate: 2

Gate ID: 3
Lock Status: 3
Prev Gate: 2



In [6]:
class FM_algo:
    def __init__(self, circuit_folder_path):
        self.circuit_info = self.read_circuit(circuit_folder_path)
        self.circuit_connections = self.circuit_info['netD_nets']
        self.bucket_gains1 = dict()
        self.bucket_gains2 = dict()
        self.vertex = dict()
        self.fd_algo()

    def fd_algo(self):
        #partition circuit
        self.partition1, self.partition2 = self.partition_circuit()
        #initialize buckets
        self.bucket_gains1 = self.initialize_buckets(self.partition1, self.bucket_gains1)
        self.bucket_gains2 = self.initialize_buckets(self.partition2, self.bucket_gains2)

        ##TODO: Begin Fiduccia-Mattheyses Algorithm below
        pass
   
    def partition_circuit(self):
        circuit_nodes = list(self.circuit_connections.keys())
        random.shuffle(circuit_nodes)
        partition = len(circuit_nodes) // 2

        partition1 = circuit_nodes[:partition]
        partition2 = circuit_nodes[partition:] 
        return partition1, partition2
    
    def calculate_gain(self, gate):
        external_sum = 0
        internal_sum = 0
        if gate in self.partition1:
            current_partition = self.partition1
        else:
            current_partition = self.partition2

        for next_gate in self.circuit_connections[gate]:
            if next_gate in current_partition:
                internal_sum += 1
            else:
                external_sum += 1
        gain = external_sum - internal_sum
        return gain
        
    def initialize_buckets(self, partition, bucket):
        for gate in partition:
            gain = self.calculate_gain(gate)
            current_gate = gate_dlinkedList(gate)
            if gain not in list(bucket.keys()):
                bucket[gain] = current_gate #gate must be doubly linked list OOC
            else:
                bucket[gain] = bucket[gain].set_next(current_gate)
            #add gate node to vertex bucket for lookup
            self.vertex[gate] = bucket[gain]
        return bucket

    def read_circuit(self, circuit_folder_path):
        net_file = None
        netD_file = None
        are_file = None
        for file in os.listdir(circuit_folder_path):
            if file.endswith('.net'):
                net_file = os.path.join(circuit_folder_path, file)
            if file.endswith('.netD'):
               netD_file = os.path.join(circuit_folder_path ,file)
            if file.endswith('.are'):
               are_file = os.path.join(circuit_folder_path, file)
    

    ## UPDATE 2/21 01:00 JIMMY
        if net_file and netD_file and are_file:
            self.circuit_data = self.parse_net_files(net_file, netD_file, are_file)
            return self.circuit_data
        else:
            print("Error Message: Missing Files")

    def parse_net_files(self, net_file, netD_file, are_file):
        # network information
        nets, connections, nodes, modules, pad_offset = self.read_net_file(net_file)
        # module areas
        areas = self.read_are_file(are_file)
        # network information with pin directions
        netD_nets = self.read_netD_file(netD_file)

        circuit_data = {
            'nets': nets,
            'areas': areas,
            'netD_nets': netD_nets,
            'connections': int(connections),
            'nodes': int(nodes),
            'modules': int(modules),
            'pad_offset': int(pad_offset)
        }
        return circuit_data
    
    def read_net_file(self, circuit_folder_path):
        with open(circuit_folder_path, 'r') as file:
            lines = file.readlines()
        circuit_info = tuple(item.replace('\n', '') for item in lines[0:5])
        #Ex: ('0', '13', '5', '7', '3')
        _, connections, nodes, modules, pad_offset  = circuit_info

        lines = lines[5:]
        net = {}
        current_key = None
        for line in lines:
            tokens = line.split()
            if tokens[1] == 's':
                current_key = tokens[0]
                net[current_key] = []
            elif tokens[1] == 'l':
                net[current_key].append(tokens[0])
    
        return net, connections, nodes, modules, pad_offset


    def read_are_file(self, circuit_folder_path):
        with open(circuit_folder_path, 'r') as file:
            lines = file.readlines()
        module_areas = {}
        for line in lines:
            module_id, area = line.split()
            module_areas[module_id] = int(area)
        return module_areas

    def read_netD_file(self, circuit_folder_path):
        with open(circuit_folder_path, 'r') as file:
            lines = file.readlines()

        lines = lines[5:]
        nets = {}
        current_key = None
        for line in lines:
            tokens = line.split()
            if tokens[1] == 's':
                current_key = tokens[0]
                nets[current_key] = []
            elif tokens[1] == 'l':
                nets[current_key].append(tokens[0])
        return nets

        ## bi-directional
        # bi_direct = {}
        # for line in lines:
        #     tokens = line.split()
        #     if tokens[1] == 's':
        #         current_key = tokens[0]
        #         nets[current_key] = []
        #     elif tokens[1] == 'l':
        #         if tokens[2] == 'B':
        #             if current_key not in bi_direct:
        #                 bi_direct[current_key] = [tokens[0]]
        #             else:
        #                 bi_direct[current_key].append(tokens[0])
        #         else:
        #             nets[current_key].append(tokens[0])

        # for key, values in bi_direct.items():
        #     if key in nets:
        #         nets[key].extend(values)
        #     else:
        #         nets[key] = values

        # for key, values in bi_direct.items():
        #     for value in values:
        #         if value in nets:
        #             nets[value].append(key)
        #         else:
        #             nets[value] = [key]

        # return nets                

In [7]:
circuit_folder_path = "test_simple"
fm = FM_algo(circuit_folder_path)

print('left partition gains', fm.bucket_gains1)
print('right partition gains', fm.bucket_gains2)
print('vertex', fm.vertex)

left partition gains {2: <__main__.gate_dlinkedList object at 0x000002067C2350D0>, 1: <__main__.gate_dlinkedList object at 0x000002067C235340>}
right partition gains {1: <__main__.gate_dlinkedList object at 0x000002067C35D8B0>, 2: <__main__.gate_dlinkedList object at 0x000002067C35DEE0>}
vertex {'a0': <__main__.gate_dlinkedList object at 0x000002067C15F8E0>, 'a4': <__main__.gate_dlinkedList object at 0x000002067C235340>, 'a1': <__main__.gate_dlinkedList object at 0x000002067C2350D0>, 'a3': <__main__.gate_dlinkedList object at 0x000002067C35D9D0>, 'p1': <__main__.gate_dlinkedList object at 0x000002067C35DEE0>, 'a2': <__main__.gate_dlinkedList object at 0x000002067C35D8B0>}


In [8]:
#### TEST FUNCTION ####
#### inputs from https://vlsicad.ucsd.edu/UCLAWeb/cheese/ispd98.html#Benchmark%20File%20Format ####
#### example with 4 cells, 3 pads, 5 nets and 13 pins ####
circuit_folder_path = "test_simple"
fm = FM_algo(circuit_folder_path)
circuit_data = fm.read_circuit(circuit_folder_path)
print(circuit_data)


{'nets': {'p1': ['a0', 'a1'], 'a0': ['a2', 'a3'], 'a1': ['a2', 'a3'], 'a2': ['p2'], 'a3': ['p3']}, 'areas': {'a0': 1, 'a1': 3, 'a2': 4, 'a3': 2, 'p1': 0, 'p2': 0, 'p3': 0}, 'netD_nets': {'p1': ['a0', 'a1'], 'a0': ['a2', 'a3'], 'a1': ['a2', 'a3'], 'a2': ['p2'], 'a3': ['p3'], 'a4': ['p4']}, 'connections': 13, 'nodes': 5, 'modules': 7, 'pad_offset': 3}


In [9]:
circuit_data['netD_nets']

{'p1': ['a0', 'a1'],
 'a0': ['a2', 'a3'],
 'a1': ['a2', 'a3'],
 'a2': ['p2'],
 'a3': ['p3'],
 'a4': ['p4']}

In [10]:
circuit_data['nets']

{'p1': ['a0', 'a1'],
 'a0': ['a2', 'a3'],
 'a1': ['a2', 'a3'],
 'a2': ['p2'],
 'a3': ['p3']}

In [11]:
circuit_data['areas']

{'a0': 1, 'a1': 3, 'a2': 4, 'a3': 2, 'p1': 0, 'p2': 0, 'p3': 0}

In [12]:
def main():
    pass

if __name__ == "__main__":
    main()

In [13]:
print(circuit_data['areas'])
print(circuit_data['netD_nets'])
print(circuit_data['nets'])

{'a0': 1, 'a1': 3, 'a2': 4, 'a3': 2, 'p1': 0, 'p2': 0, 'p3': 0}
{'p1': ['a0', 'a1'], 'a0': ['a2', 'a3'], 'a1': ['a2', 'a3'], 'a2': ['p2'], 'a3': ['p3'], 'a4': ['p4']}
{'p1': ['a0', 'a1'], 'a0': ['a2', 'a3'], 'a1': ['a2', 'a3'], 'a2': ['p2'], 'a3': ['p3']}
